<a href="https://colab.research.google.com/github/JoeC5/Synthetic-Mortgage-DB-review/blob/main/mtg_dbcreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following is to build a synthetic 30 year mortgage loan portfolio. The data will include:
- customer ID
- demographic information - age, income, credit score
- property information - value, location, type
- loan details - amount interest rate, LTV, origination date, FHA, Conventional
- loan status - current, delinquent default

In [ ]:
!pip install faker

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from faker import Faker


In [ ]:
#set random seed for reproducibility
np.random.seed(42)
random.seed(42)
fake=Faker()
Faker.seed(42)

In [ ]:
#define the number of customers
num_customers=1000

In [ ]:
def generate_mortgage_data(num_customers):
  #initialize lists to store customer data
  customer_ids = range(1001, 1001 + num_customers)

  #customer demographics
  ages = np.random.normal(45, 10, num_customers).astype(int)
  ages = np.clip(ages, 22, 75)

  incomes = np.random.lognormal(mean=11, sigma=0.5, size=num_customers)  #log normal for income distribution
  incomes = np.round(incomes).astype(int)

  credit_scores = np.random.normal(720, 80, num_customers).astype(int)
  credit_scores = np.clip(credit_scores, 580, 850)  #clip to valid FICO score

  #property information
  property_values = np.random.lognormal(mean=12.8, sigma=0.5, size=num_customers)
  property_values = (np.round(property_values/5000)* 5000).astype(int)

  property_types = np.random.choice(['Single Family', 'Condo', 'Townhouse'],
                                    size=num_customers,
                                    p=[0.7, 0.2, 0.1])


  all_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

  #State weights are based population of each state - a percentage of total US
  #population. I pulled this data and calculated in seperate Excel spreadsheet.
  state_weights = {
      'AL':0.015, 'AK':0.002, 'AZ':0.022, 'AR':0.01, 'CA':0.117, 'CO':0.018,
      'CT':0.011, 'DE':0.004, 'FL':0.07, 'GA':0.033, 'HI':0.004, 'ID':0.006,
      'IL':0.038, 'IN':0.02, 'IA':0.009, 'KS':0.009, 'KY':0.012, 'LA':0.013,
      'ME':0.004, 'MD':0.018, 'MA':0.021, 'MI':0.03, 'MN':0.017, 'MS':0.009,
      'MO':0.018, 'MT':0.003, 'NE':0.006, 'NV':0.009, 'NH':0.004, 'NJ':0.027,
      'NM':0.006, 'NY':0.059, 'NC':0.033, 'ND':0.002, 'OH':0.035, 'OK':0.012,
      'OR':0.012, 'PA':0.039, 'RI':0.003, 'SC':0.016, 'SD':0.003, 'TN':0.021,
      'TX':0.094, 'UT':0.01, 'VT':0.002, 'VA':0.026, 'WA':0.023, 'WV':0.005,
      'WI':0.017, 'WY':0.002
      }
  # Ensure weights are normalized
  weights_array = np.array([state_weights[state] for state in all_states])
  probabilities = weights_array / weights_array.sum()  # Ensures sum = 1

  # Generate synthetic states
  num_customers = 1000
  states = np.random.choice(all_states, size=num_customers, p=probabilities)

  # Loan information - using LTV of 60% to 95%
  loan_amounts = property_values * np.random.uniform(0.60, 0.95, num_customers)
  loan_amounts = np.round(loan_amounts / 1000) * 1000  # Round to nearest $1000

  down_payments = property_values - loan_amounts

  # Interest rates based on credit score - usig a credit score range of 580 to 850
  #starting with a "base rate" 8.5% for a 580 credit score - as credit scores rise the interest rate decreases
  interest_rates = 8.0 - (credit_scores - 580) * (3.5 / (850-580)) + np.random.normal(0, 0.2, num_customers)
  interest_rates = np.clip(interest_rates, 3.0, 8.5)
  interest_rates = np.round(interest_rates * 100) / 100  # Round to 2 decimal places

  # Loan terms (years)
  loan_terms = np.random.choice([20, 30], size=num_customers, p=[0.1, 0.90])


  #origination dates (within the past 10 years)
  today = datetime.now()
  max_days = 365 * 10   #10 year
  origination_days_ago = np.random.randint(0, max_days, num_customers)
  origination_dates = [(today - timedelta(days=int(days))).strftime('%Y-%m-%d') for days in origination_days_ago]

 # Calculate monthly payments (simplified)
  monthly_rate = interest_rates / 100 / 12
  loan_terms_months = loan_terms * 12
  monthly_payments = loan_amounts * (monthly_rate * (1 + monthly_rate) ** loan_terms_months) / ((1 + monthly_rate) ** loan_terms_months - 1)
  monthly_payments = np.round(monthly_payments, 2)

  # Current balance based on how long ago the loan started
  # Simple calculation - not accounting for amortization properly
  payments_made = origination_days_ago / 30
  total_payments = loan_terms_months
  remaining_principal_ratio = 1 - (payments_made / total_payments) * 0.5  # Simplified calculation
  current_balances = loan_amounts * np.clip(remaining_principal_ratio, 0, 1)
  current_balances = np.round(current_balances, 2)

  # Loan status
  # Most loans are current, some are delinquent, a few are in default
  delinquency_prob = 0.08  # 8% chance of delinquency
  default_prob = 0.03      # 3% chance of default

  status_random = np.random.random(num_customers)
  loan_status = ['Current'] * num_customers
  for i in range(num_customers):
      if status_random[i] < default_prob:
          loan_status[i] = 'Default'
      elif status_random[i] < default_prob + delinquency_prob:
          loan_status[i] = 'Delinquent'

  # Days delinquent for non-current loans
  days_delinquent = [0] * num_customers
  for i in range(num_customers):
      if loan_status[i] == 'Delinquent':
          days_delinquent[i] = np.random.randint(30, 120)
      elif loan_status[i] == 'Default':
          days_delinquent[i] = np.random.randint(120, 500)

  # Generate customer names
  first_names = [fake.first_name() for _ in range(num_customers)]
  last_names = [fake.last_name() for _ in range(num_customers)]

  # Generate addresses
  addresses = [fake.street_address() for _ in range(num_customers)]
  cities = [fake.city() for _ in range(num_customers)]
  zip_codes = [fake.zipcode() for _ in range(num_customers)]

  # Create the dataframe
  mortgage_data = pd.DataFrame({
      'customer_id': customer_ids,
      'first_name': first_names,
      'last_name': last_names,
      'age': ages,
      'annual_income': incomes,
      'credit_score': credit_scores,
      'address': addresses,
      'city': cities,
      'state': states,
      'zip_code': zip_codes,
      'property_type': property_types,
      'property_value': property_values,
      'loan_amount': loan_amounts,
      'down_payment': down_payments,
      'interest_rate': interest_rates,
      'loan_term_years': loan_terms,
      'origination_date': origination_dates,
      'monthly_payment': monthly_payments,
      'current_balance': current_balances,
      'loan_status': loan_status,
      'days_delinquent': days_delinquent
  })

  return mortgage_data

In [ ]:
# Generate the data
mortgage_df = generate_mortgage_data(num_customers)

In [ ]:
#Preview the data
print(mortgage_df.head())
print("\nDataset shape:", mortgage_df.shape)
print("\nSummary statistics:")
print(mortgage_df.describe())

   customer_id first_name last_name  age  annual_income  credit_score  \
0         1001   Danielle    Garcia   49         120533           665   
1         1002      Angel  Jennings   43          95065           708   
2         1003     Joshua  Stephens   51          61686           656   
3         1004    Jeffrey    Levine   60          43327           695   
4         1005       Jill    Howard   42          84890           580   

                     address              city state zip_code  ...  \
0             836 Hill Point        Piercestad    CO    51491  ...   
1             687 Teresa Row      South Rachel    MI    74000  ...   
2  566 Kevin Ferry Suite 334          Luisstad    ID    95622  ...   
3            0866 Paul Glens  Lake Nicolemouth    AZ    43569  ...   
4            1751 Brett Lane         Perezfurt    IL    23731  ...   

  property_value  loan_amount  down_payment  interest_rate  loan_term_years  \
0         140000     115000.0       25000.0           7.08   

In [ ]:
#export to CSV
mortgage_df.to_csv('synthetic_mortgage_data.csv', index=False)
print("n\Data saved to 'synthetic_mortgage_data.csv'")

n\Data saved to 'synthetic_mortgage_data.csv'
